In [121]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Labelling of train and test data

In [122]:
import os
df_train=pd.read_excel('iris_train_J.xlsx',sheet_name='Train',engine='openpyxl')
df_test=pd.read_excel('iris_test_J.xlsx',engine='openpyxl')

In [123]:
df_train.shape

(22711, 2)

In [124]:
df_test.shape

(3505, 2)

In [125]:
import re
# Dictionary of english Contractions - move 
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions
df_train['utterance']=df_train['utterance'].apply(lambda x:expand_contractions(x))

## Cleaning for train data

1. Current one arjun excluded above 30 ones
2. We should omit utternce>30, or if you want, truncate it and take the latter portion to preserve the valueable information

In [126]:
ng=pd.read_csv('ngrams.csv').ngram.dropna()
ng1=[i.split() for i in ng.tolist()]
ngrams=[i for j in ng1 for i in j]

In [135]:
def lametize(text):
    #stopwords=nltk.corpus.stopwords.words('english')
    #text=[word for word in tokenize(text) if word not in stopwords]
    k=''
    for j in tokenize(text):
        for i in ngrams:
            if j in i:
                k+=' '+j
            else:
                k+=' '+lemm.lemmatize(j)
            break
    #text=[lemm.lemmatize(word) for word in tokenize(text) if word not in ngrams]
    return k.strip()

In [128]:
import re
import string
def clean_text(text):
    text=str(text)
    text=text.lower()
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    split=re.split("\W+",text)
    return split

import nltk
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()
# def remove_stopwords(text):
#     stopwords=nltk.corpus.stopwords.words('english')
#     text=[lemm.lemmatize(word) for word in tokenize(text) if word not in stopwords]
#     return ' '.join(text)


df_train['utterance']=df_train['utterance'].apply(lambda x: clean_text(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_train['utterance']=df_train['utterance'].str.replace(".", "")
df_train['utterance']=df_train['utterance'].apply(lambda x: remove_punctuation(x))

df_train['utterance']=df_train['utterance'].apply(lambda x: lametize(x))
df_train['utterance']=df_train['utterance'].str.split().str[-30:].str.join(' ')
df_train['intent']=df_train['intent'].astype(str).str.lower()

# selecting on > 2 len sentence utterances
df_trainutt=df_train["utterance"].values
df_trainutt
s = pd.Series(df_trainutt)
df_train["utt"]=s.str.split().to_frame()
type(s.str.split().to_frame())
df_train['Length'] = df_train['utt'].str.len()
df_train
#last step
df_train.query('Length > 2', inplace = True) 
df_train=df_train[["utterance","intent"]]
df_train

,utterance,intent
0,how to apply for 1bank account password reset,access_management.reset_1bank_password
1,i am not able to see some application in the d...,access_management.missing_application_in_uams
2,is bulk request available,access_management.process_bulk_request_bulk_ap...
3,why are some application missing,access_management.missing_application_in_uams
4,1bank id password reset in uams,access_management.reset_1bank_password
...,...,...
22706,update of the status of software request,computer.software_installation
22707,update of my software request,computer.software_installation
22708,current status of existing software request,computer.software_installation
22709,check status of existing software request,computer.software_installation


In [129]:
# import spacy
# nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
# nlp.max_length=5000000
# # nlp = spacy.load('C:/Users/thiyaneswaranm/Documents/New folder/model_api_gold/en_core_web_sm-2.2.5/en_core_web_sm/en_core_web_sm-2.2.5',disable=['ner','parser'])
# def lemmatize_sent(text):
#     doc = nlp(text)
#     lemm_word=[]
#     for token in doc:
#         if (token.lemma_.casefold())!=("-pron-"):
#     #         print(token, token.lemma_,type(token.lemma_))
#             lemm_word.append(token.lemma_)
#         else:
#             lemm_word.append(token)
#     listToStr = ' '.join([str(elem) for elem in lemm_word]) 
#     return str(listToStr.lower())
# print(lemmatize_sent("my am able min to"))
# df_train['utterance']=df_train['utterance'].apply(lemmatize_sent)

## Cleaning for test data

In [130]:
import re
import string
def clean_text(text):
    text=str(text)
    
    
    text=text.lower()
    #text=re.sub('\n',' ',text)
    #text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    split=re.split("\W+",text)
    return split

import nltk
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()


# def remove_stopwords(text):
#     stopwords=nltk.corpus.stopwords.words('english')
#     text=[word for word in tokenize(text) if word not in stopwords]
#     text=[lemm.lemmatize(word) for word in text if word not in ngrams]
#     return ' '.join(text)


df_test['utterance']=df_test['utterance'].apply(lambda x: clean_text(x))
df_test['utterance']=df_test['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_test['utterance']=df_test['utterance'].str.replace(".", "")
df_test['utterance']=df_test['utterance'].apply(lambda x: remove_punctuation(x))
df_test['utterance']=df_test['utterance'].apply(lambda x: lametize(x))
df_test['intent']=df_test['intent'].astype(str).str.lower()

df_test['utterance']=df_test['utterance'].str.split().str[-30:].str.join(' ')

In [131]:
df_test

,utterance,intent
0,access management apply access functional id i...,apply access for funcional account in uams
1,access management how to perform delegation in...,approver on leave for uams
2,access management who is my authoriser to appr...,can i select my approver in uams
3,access management track request process flow,can i track my request process
4,access management update access matrix for app...,can i update the access matrix for applicatio...
...,...,...
3500,microsoft office 365 team drive mapping,map a team drive to my machine
3501,microsoft office 365 unable to view the image ...,something wrong with the images in my email
3502,mobility suite device not registered under app...,registration of non dep ipad
3503,mobility suite out of office on on on blackber...,need set up bb work out of office


In [132]:
# selecting on > 2 len sentence utterances
# df_testutt=df_test["utterance"].values
# df_testutt
# s = pd.Series(df_testutt)
# df_test["utt"]=s.str.split().to_frame()
# type(s.str.split().to_frame())
# df_test['Length'] = df_test['utt'].str.len()
# df_test
# #last step
# df_test.query('Length > 2', inplace = True) 
# df_test=df_test[["utterance","intent"]]
# df_test['utterance'][df_test['utterance']=='']='NONE'
# df_test

1. remove the c intent (less than 15 utterance)(due to class imbalance). when you find c intent, you dont call the prediction and ignore it and mark it as NA

In [64]:
# more_than_15=df_train['intent'].value_counts()[df_train['intent'].value_counts()>15].index
# df_test['intent'][df_test['intent'].isin(more_than_15)]='NA'

In [65]:
df_test

,utterance,intent
0,access management apply access functional id uams,apply access for funcional account in uams
1,access management perform delegation uams,approver on leave for uams
2,access management authoriser approve request uams,can i select my approver in uams
3,access management track request process flow,can i track my request process
4,access management update access matrix applica...,can i update the access matrix for applicatio...
...,...,...
3500,microsoft office 365 team drive mapping,map a team drive to my machine
3501,microsoft office 365 unable view image outlook,something wrong with the images in my email
3502,mobility suite device registered apple device ...,registration of non dep ipad
3503,mobility suite office blackberry work,need set up bb work out of office


## Labelling

In [66]:
#df=pd.concat([df_train[['utterance','intent']],df_test[['utterance','intent']]])
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

In [67]:
df_test=pd.merge(df_test,mapping,on='intent',how='left').rename(columns={'mapping':'intent1'})

df_test['intent1']=df_test['intent1'].fillna(0).astype(int)

In [68]:
df_test

,utterance,intent,intent1
0,access management apply access functional id uams,apply access for funcional account in uams,0
1,access management perform delegation uams,approver on leave for uams,0
2,access management authoriser approve request uams,can i select my approver in uams,0
3,access management track request process flow,can i track my request process,0
4,access management update access matrix applica...,can i update the access matrix for applicatio...,0
...,...,...,...
3500,microsoft office 365 team drive mapping,map a team drive to my machine,0
3501,microsoft office 365 unable view image outlook,something wrong with the images in my email,0
3502,mobility suite device registered apple device ...,registration of non dep ipad,0
3503,mobility suite office blackberry work,need set up bb work out of office,0


## Save the files

In [69]:
df_train.to_excel('train_iris_fullcleaned1.xlsx',index=None)

writer=pd.ExcelWriter('test_iris_fullcleaned1.xlsx',engine='xlsxwriter')

df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)

writer.save()